In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("amp.txt", sep="\t")

In [3]:
df

,sample,SiO2,TiO2,Al2O3,FeO,MnO,MgO,CaO,Na2O,K2O
0,amp,43.55,1.28,13.62,9.62,0.0,14.53,10.25,2.51,0.64
1,amp,52.50,0.00,4.40,20.40,0.3,18.00,0.50,0.10,0.00
2,amp,49.01,0.20,14.91,0.55,0.0,28.62,0.23,1.46,0.00


In [4]:
from Amphibole import normalization

In [5]:
an_df, cat_df = normalization(df)

In [6]:
an_df

,Si_anion,Ti_anion,Al_anion,Fe2_anion,Mn_anion,Mg_anion,Ca_anion,Na_anion,K_anion,total_anion
0,1.449630,0.032049,0.400745,0.133897,0.000000,0.360507,0.182777,0.040499,0.006794,2.606896
1,1.747545,0.000000,0.129462,0.283939,0.004229,0.446601,0.008916,0.001613,0.000000,2.622306
2,1.631375,0.005008,0.438701,0.007655,0.000000,0.710096,0.004101,0.023557,0.000000,2.820492


In [7]:
cat_df

,Si_cation,Ti_cation,Al_cation,Fe2+_cation,Mn_cation,Mg_cation,Ca_cation,Na_cation,K_cation,total_cationFM,total_cationFM+Ca
0,0.724815,0.016024,0.267163,0.133897,0.000000,0.360507,0.182777,0.080997,0.013589,1.502406,1.685182
1,0.873772,0.000000,0.086308,0.283939,0.004229,0.446601,0.008916,0.003227,0.000000,1.694850,1.703766
2,0.815687,0.002504,0.292467,0.007655,0.000000,0.710096,0.004101,0.047114,0.000000,1.828410,1.832511


In [8]:
def teste(dataframe):
    an, cat = normalization(dataframe)

    def normalization_13(an, cat):
        """S13 normalized based on anions"""

        anion = an
        cation = cat
        df_s13 = pd.DataFrame()

        df_s13['Si_S13'] = 13 * anion['Si_anion'] / cation['total_cationFM']
        df_s13['Ti_S13'] = 13 * anion['Ti_anion'] / cation['total_cationFM']
        df_s13['Al_S13'] = 13 * anion['Al_anion'] / cation['total_cationFM']
        df_s13['Fe2+_S13'] = 13 * anion['Fe2_anion'] / cation['total_cationFM']
        df_s13['Fe3+_S13'] = 0
        df_s13['Mn_S13'] = 13 * anion['Mn_anion'] / cation['total_cationFM']
        df_s13['Mg_S13'] = 13 * anion['Mg_anion'] / cation['total_cationFM']
        df_s13['Ca_S13'] = 13 * anion['Ca_anion'] / cation['total_cationFM']
        df_s13['Na_S13'] = 13 * anion['Na_anion'] / cation['total_cationFM']
        df_s13['K_S13'] = 13 * anion['K_anion'] / cation['total_cationFM']
        df_s13['total_S13'] = df_s13.sum(axis='columns')

        return df_s13

    def normalization_15(an, cat):
        """S13 normalized based on anions"""

        anion = an
        cation = cat
        df_s15 = pd.DataFrame()

        df_s15['Si_S15'] = 15 * anion['Si_anion'] / cation['total_cationFM+Ca']
        df_s15['Ti_S15'] = 15 * anion['Ti_anion'] / cation['total_cationFM+Ca']
        df_s15['Al_S15'] = 15 * anion['Al_anion'] / cation['total_cationFM+Ca']
        df_s15['Fe2+_S15'] = 15 * anion['Fe2_anion'] / cation['total_cationFM+Ca']
        df_s15['Fe3+_S15'] = 0
        df_s15['Mn_S15'] = 15 * anion['Mn_anion'] / cation['total_cationFM+Ca']
        df_s15['Mg_S15'] = 15 * anion['Mg_anion'] / cation['total_cationFM+Ca']
        df_s15['Ca_S15'] = 15 * anion['Ca_anion'] / cation['total_cationFM+Ca']
        df_s15['Na_S15'] = 15 * anion['Na_anion'] / cation['total_cationFM+Ca']
        df_s15['K_S15'] = 15 * anion['K_anion'] / cation['total_cationFM+Ca']
        df_s15['total_S15'] = df_s15.sum(axis='columns')

        return df_s15
    
    s13, s15 = normalization_13(an, cat), normalization_15(an, cat)

    def oxid(s13, s15):
        #Fe3+ calculation
        s13['Fe3+_S13'] = 2 * (23 - s13['total_S13'])
        for value in s13['Fe3+_S13']:
            if value < 0:
                s15['Fe3+_S15'] = 0
            else:
                s15['Fe3+_S15'] = value

        s15['Fe3+_S15'] = 2 * (23 - s15['total_S15'])
        for value in s15['Fe3+_S15']:
            if value < 0:
                s15['Fe3+_S15'] = 0
            else:
                s15['Fe3+_S15'] = value

        #Fe3+ and Fe2+ calculation S13
        for index, value in enumerate(s13['Fe3+_S13']):
            if value > s13['Fe2+_S13'][index]:
                s13['Fe3+_S13'][index] = s13['Fe2+_S13'][index]
            else:
                s13['Fe3+_S13'][index]

        s13['Fe2+_S13'] = s13['Fe2+_S13'] - s13['Fe3+_S13']

        #Fe3+ and Fe2+ calculation S15
        for index, value in enumerate(s15['Fe3+_S15']):
            if value > s15['Fe2+_S15'][index]:
                s15['Fe3+_S15'][index] = s15['Fe2+_S15'][index]
            else:
                s15['Fe3+_S15'][index]


        s15['Fe2+_S15'] = s15['Fe2+_S15'] - s15['Fe3+_S15']
        
        return s13, s15

    s13_ox, s15_ox = oxid(s13, s15)
    return s13_ox, s15_ox


In [9]:
s13, s15 = teste(df)

In [10]:
s13

,Si_S13,Ti_S13,Al_S13,Fe2+_S13,Fe3+_S13,Mn_S13,Mg_S13,Ca_S13,Na_S13,K_S13,total_S13
0,12.543343,0.277309,3.467559,0.272426,0.886155,0.000000,3.119387,1.581527,0.350426,0.05879,22.556923
1,13.404184,0.000000,0.993014,0.000000,2.177896,0.032438,3.425564,0.068388,0.012376,0.00000,20.113860
2,11.599081,0.035604,3.119163,0.000000,0.054429,0.000000,5.048787,0.029160,0.167490,0.00000,20.053714


In [11]:
s15

,Si_S15,Ti_S15,Al_S15,Fe2+_S15,Fe3+_S15,Mn_S15,Mg_S15,Ca_S15,Na_S15,K_S15,total_S15
0,12.903322,0.285268,3.567074,1.191830,0,0.000000,3.208910,1.626915,0.360483,0.060477,23.204280
1,15.385429,0.000000,1.139789,2.499807,0,0.037233,3.931890,0.078496,0.014205,0.000000,23.086849
2,13.353601,0.040990,3.590979,0.062662,0,0.000000,5.812485,0.033571,0.192825,0.000000,23.087114
